# Create anchor boxes

This is the **SsdAnchorsCalculator** stage from the MediaPipe graph. It computes a list of anchor boxes. This only needs to be done once, so we will store these anchors into a lookup table.

In [1]:
import numpy as np

These are the options from [face_detection_mobile_gpu.pbtxt](https://github.com/google/mediapipe/blob/master/mediapipe/graphs/face_detection/face_detection_mobile_gpu.pbtxt).

To understand what these options mean, see [ssd_anchors_calculator.proto](https://github.com/google/mediapipe/blob/master/mediapipe/calculators/tflite/ssd_anchors_calculator.proto).

In [2]:
anchor_options = {
    "num_layers": 4,
    "min_scale": 0.1484375,
    "max_scale": 0.75,
    "input_size_height": 128,
    "input_size_width": 128,
    "anchor_offset_x": 0.5,
    "anchor_offset_y": 0.5,
    "strides": [8, 16, 16, 16],
    "aspect_ratios": [1.0],
    "reduce_boxes_in_lowest_layer": False,
    "interpolated_scale_aspect_ratio": 1.0,
    "fixed_anchor_size": True,
}

This is a literal translation of [ssd_anchors_calculator.cc](https://github.com/google/mediapipe/blob/master/mediapipe/calculators/tflite/ssd_anchors_calculator.cc):

In [3]:
def calculate_scale(min_scale, max_scale, stride_index, num_strides):
    return min_scale + (max_scale - min_scale) * stride_index / (num_strides - 1.0)


def generate_anchors(options):
    strides_size = len(options["strides"])
    assert options["num_layers"] == strides_size

    anchors = []
    layer_id = 0
    while layer_id < strides_size:
        anchor_height = []
        anchor_width = []
        aspect_ratios = []
        scales = []

        # For same strides, we merge the anchors in the same order.
        last_same_stride_layer = layer_id
        while (last_same_stride_layer < strides_size) and \
              (options["strides"][last_same_stride_layer] == options["strides"][layer_id]):
            scale = calculate_scale(options["min_scale"],
                                    options["max_scale"],
                                    last_same_stride_layer,
                                    strides_size)

            if last_same_stride_layer == 0 and options["reduce_boxes_in_lowest_layer"]:
                # For first layer, it can be specified to use predefined anchors.
                aspect_ratios.append(1.0)
                aspect_ratios.append(2.0)
                aspect_ratios.append(0.5)
                scales.append(0.1)
                scales.append(scale)
                scales.append(scale)                
            else:
                for aspect_ratio in options["aspect_ratios"]:
                    aspect_ratios.append(aspect_ratio)
                    scales.append(scale)

                if options["interpolated_scale_aspect_ratio"] > 0.0:
                    scale_next = 1.0 if last_same_stride_layer == strides_size - 1 \
                                     else calculate_scale(options["min_scale"],
                                                          options["max_scale"],
                                                          last_same_stride_layer + 1,
                                                          strides_size)
                    scales.append(np.sqrt(scale * scale_next))
                    aspect_ratios.append(options["interpolated_scale_aspect_ratio"])

            last_same_stride_layer += 1

        for i in range(len(aspect_ratios)):
            ratio_sqrts = np.sqrt(aspect_ratios[i])
            anchor_height.append(scales[i] / ratio_sqrts)
            anchor_width.append(scales[i] * ratio_sqrts)            
            
        stride = options["strides"][layer_id]
        feature_map_height = int(np.ceil(options["input_size_height"] / stride))
        feature_map_width = int(np.ceil(options["input_size_width"] / stride))

        for y in range(feature_map_height):
            for x in range(feature_map_width):
                for anchor_id in range(len(anchor_height)):
                    x_center = (x + options["anchor_offset_x"]) / feature_map_width
                    y_center = (y + options["anchor_offset_y"]) / feature_map_height

                    new_anchor = [x_center, y_center, 0, 0]
                    if options["fixed_anchor_size"]:
                        new_anchor[2] = 1.0
                        new_anchor[3] = 1.0
                    else:
                        new_anchor[2] = anchor_width[anchor_id]
                        new_anchor[3] = anchor_height[anchor_id]
                    anchors.append(new_anchor)

        layer_id = last_same_stride_layer

    return anchors

In [4]:
anchors = generate_anchors(anchor_options)

assert len(anchors) == 896

Each anchor is `[x_center, y_center, width, height]` in normalized coordinates. For our use case, the width and height are always 1.

In [5]:
anchors[:10]

[[0.03125, 0.03125, 1.0, 1.0],
 [0.03125, 0.03125, 1.0, 1.0],
 [0.09375, 0.03125, 1.0, 1.0],
 [0.09375, 0.03125, 1.0, 1.0],
 [0.15625, 0.03125, 1.0, 1.0],
 [0.15625, 0.03125, 1.0, 1.0],
 [0.21875, 0.03125, 1.0, 1.0],
 [0.21875, 0.03125, 1.0, 1.0],
 [0.28125, 0.03125, 1.0, 1.0],
 [0.28125, 0.03125, 1.0, 1.0]]

Run the "FaceDetectionConfig" test case from the MediaPipe repo:

In [6]:
anchor_options_test = {
    "num_layers": 5,
    "min_scale": 0.1171875,
    "max_scale": 0.75,
    "input_size_height": 256,
    "input_size_width": 256,
    "anchor_offset_x": 0.5,
    "anchor_offset_y": 0.5,
    "strides": [8, 16, 32, 32, 32],
    "aspect_ratios": [1.0],
    "reduce_boxes_in_lowest_layer": False,
    "interpolated_scale_aspect_ratio": 1.0,
    "fixed_anchor_size": True,
}

anchors_test = generate_anchors(anchor_options_test)
anchors_golden = np.loadtxt("./mediapipe/mediapipe/calculators/tflite/testdata/anchor_golden_file_0.txt")

assert len(anchors_test) == len(anchors_golden)
print("Number of errors:", (np.abs(anchors_test - anchors_golden) > 1e-5).sum())

Number of errors: 0


Run the "MobileSSDConfig" test case from the MediaPipe repo:

In [7]:
anchor_options_test = {
    "num_layers": 6,
    "min_scale": 0.2,
    "max_scale": 0.95,
    "input_size_height": 300,
    "input_size_width": 300,
    "anchor_offset_x": 0.5,
    "anchor_offset_y": 0.5,
    "strides": [16, 32, 64, 128, 256, 512],
    "aspect_ratios": [1.0, 2.0, 0.5, 3.0, 0.3333],
    "reduce_boxes_in_lowest_layer": True,
    "interpolated_scale_aspect_ratio": 1.0,
    "fixed_anchor_size": False,
}

anchors_test = generate_anchors(anchor_options_test)
anchors_golden = np.loadtxt("./mediapipe/mediapipe/calculators/tflite/testdata/anchor_golden_file_1.txt")

assert len(anchors_test) == len(anchors_golden)
print("Number of errors:", (np.abs(anchors_test - anchors_golden) > 1e-5).sum())

Number of errors: 0


Save the anchors to a file:

In [8]:
np.save("anchors.npy", anchors)